In [3]:
import pandas as pd
import re

# --- 1. Wczytaj plik ---
df = pd.read_excel("wsk_fin.xlsx")

# --- 2. Mapa zamian dla kolumny „wskaźnik" ---
mapping = {
    "EN":   "EN",
    "PEN":  "PEN",
    "GS":   "GS",
    "PNPM": "PNPM",
    "GS_I": "GS_I",
    "FIN":  "FIN",
    "PPO":  "PPO",
    "NP":   "NP",
    "OP":   "OP",
    "POS":  "POS",
    "CF":   "CF",
    "TC":   "TC",
    "OFE":  "OFE",
    "IP":   "IP",
    "DEPR": "DEPR",
    "IO":   "IO",
    "NWC":  "NWC",
    "C":    "C",
    "LTL":  "LTL",
    "STL":  "STL",
    "LTC":  "LTC",
    "STC":  "STC",
    "INV":  "INV",
    "REC":  "REC",
    "UP":   "UP"
}

# Usuń spacje i zamień GS (I) → GS_I jeśli takie występują
df["wskaźnik"] = df["wskaźnik"].str.replace("GS (I)", "GS_I", regex=False)

# Wyodrębnij skrót (część przed spacją lub całą zawartość jeśli nie ma spacji)
df["wskaźnik"] = df["wskaźnik"].str.extract(r'^(\w+)')[0]

# --- 3. Usuń kolumny "nazwa PKD" oraz "numer i nazwa PKD" ---
df = df.drop(columns=[c for c in ["nazwa PKD", "numer i nazwa PKD"] if c in df.columns])

# --- 4. Usuń wszystkie wiersze gdzie 'numer PKD' zawiera 'SEK' ---
df = df[~df["numer PKD"].astype(str).str.contains("SEK")]

# --- 5. Usuń kolumny 2005–2021 ---
years_to_drop = [str(y) for y in range(2005, 2022)]
df = df.drop(columns=[c for c in years_to_drop if c in df.columns])

# --- 6. Usuń wiersze gdzie numer PKD ma więcej niż jedną cyfrę po kropce ---
# Dozwolone: 01.1  35.3
# Niedozwolone: 01.11  35.23
pattern_valid = r"^\d{2}\.\d$"   # dwie cyfry, kropka, jedna cyfra

df = df[df["numer PKD"].astype(str).str.match(pattern_valid, na=False)]

print(df.head())

   numer PKD wskaźnik     2005     2006     2007     2008     2009     2010  \
72      01.1       EN      407      421      405      383      424      417   
73      01.1      PEN      349      372      375      331      359      372   
74      01.1       GS  2342.87  2624.12  2971.41  2903.12  3220.56  3516.73   
75      01.1     PNPM       bd  2196.49  2392.19  2266.51  2620.78  2887.85   
76      01.1     GS_I  2081.96  2263.84  2539.82   2419.1  2672.69  2913.21   

       2011     2012  ...     2015     2016     2017     2018     2019  \
72      425      368  ...      364      363      337      321      308   
73      359      337  ...      308      304      278      260      246   
74  3953.25  4229.45  ...  3803.48  3743.89   3867.9  3983.04  4097.06   
75  3128.72  3401530  ...  3229.18  3154.91  3304.88  3442.36  3598.17   
76  3269.35  3512.17  ...  3274.65  3219.21  3392.05  3459.63  3630.23   

       2020     2021     2022     2023     2024  
72      275      267      253 